In [ ]:
!pip install ipython-autotime
# !pip install -U scikit-learn
%load_ext autotime

time: 1.37 ms (started: 2021-01-23 13:18:50 +00:00)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 6.5 ms (started: 2021-01-23 13:18:50 +00:00)


In [ ]:
!pwd
import os
os.chdir('/content/drive/My Drive/ML/ml-project-master/src/')
!pwd

/content
/content/drive/My Drive/ML/ml-project-master/src
time: 461 ms (started: 2021-01-23 13:18:50 +00:00)


In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

from utility import *

time: 3 s (started: 2021-01-23 13:18:50 +00:00)


In [ ]:
import keras.backend as K


def euclidean_distance_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))

time: 41.1 ms (started: 2021-01-23 13:18:53 +00:00)


# Import Dataset

In [ ]:
def load_cup():
    ml_cup = np.delete(np.genfromtxt('ML-CUP20-TR.csv', 
                                     delimiter=','), obj=0, axis=1)
    return ml_cup[:, :-2], ml_cup[:, -2:]

def load_cup_blind():
    return np.delete(np.genfromtxt('ML-CUP20-TS.csv',
                                   delimiter=','), obj=0, axis=1)
    
def mean_euclidean_error(y_true, y_pred):
    assert y_true.shape == y_pred.shape
    return np.mean(np.linalg.norm(y_pred - y_true, axis=1))

time: 7.29 ms (started: 2021-01-23 13:18:53 +00:00)


In [ ]:
X, y = load_cup()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

time: 419 ms (started: 2021-01-23 13:18:53 +00:00)


In [ ]:
print(X.shape)
print(y.shape)

print(X_train.shape)
print(y_train.shape)

(1524, 10)
(1524, 2)
(1219, 10)
(1219, 2)
time: 4.03 ms (started: 2021-01-23 13:18:54 +00:00)


# Grid Search - Gradient Descent

In [ ]:
txt_best_grids = "best_results_svr_cup1.txt"
grid_results_name = 'grid_results_svr_cup_v3.csv'

time: 1.64 ms (started: 2021-01-23 13:18:54 +00:00)


In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import make_scorer

param_grid = {'estimator__C': [0.1, 1, 10, 100, 1000],
              'estimator__gamma': [0.0001, 0.001, 0.005, 0.01, 0.1, 1, 3, 5],
              'estimator__epsilon': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10], 
              'estimator__kernel': ['rbf', 'linear','poly','sigmoid']} # ['rbf', 'linear','poly','sigmoid']
              

svr = MultiOutputRegressor(SVR())
neg_mean_euclidean_error = make_scorer(mean_euclidean_error, greater_is_better=False)

grid = GridSearchCV(svr, param_grid, scoring=neg_mean_euclidean_error, cv= 3, 
                    verbose=1, n_jobs=-1, return_train_score=True)
grid_result = grid.fit(X_train, y_train)


#grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, return_train_score=True, verbose=True)



Fitting 3 folds for each of 1760 candidates, totalling 5280 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 58.8min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed: 117.0min


KeyboardInterrupt: ignored

time: 2h 34min 21s (started: 2021-01-23 13:18:54 +00:00)


In [ ]:
svr.get_params().keys()

In [ ]:
import json

grid_results_name_str = f'Results wrt: {grid_results_name} \n'
grid_params_str = f'Grid on: {json.dumps(grid_result.param_grid)} \n'
grid_results_str = f'Best: {grid_result.best_score_} using {grid_result.best_params_} \n'

with open(txt_best_grids, "a") as file_object:
    file_object.write(grid_results_name_str)
    file_object.write(grid_params_str)
    file_object.write(grid_results_str)
    file_object.write('\n')

In [ ]:
print(grid_results_str)

In [ ]:
import pandas as pd

df = pd.DataFrame(grid.cv_results_)[['param_estimator__kernel', 'param_estimator__epsilon', 
                                     'param_estimator__C', 'param_estimator__gamma', 
                                     'mean_test_score', 'std_test_score', 
                                     'mean_train_score', 'std_train_score',
                                     'rank_test_score', 'mean_fit_time']].sort_values(by='rank_test_score')
df.rename(columns={'param_estimator__kernel': 'kernel',
                   'param_estimator__epsilon': 'epsilon',
                   'param_estimator__C': 'C',
                   'param_estimator__gamma': 'gamma',
                   'mean_test_score': 'mean_val_score',
                   'std_test_score':'std_val_score',
                   'rank_test_score': 'rank_val_score'}, inplace=True)
df.mean_train_score *= -1
df.mean_val_score *= -1
df

In [ ]:
df.to_csv(grid_results_name)

In [ ]:
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

# import csv

# with open(single_grid_results_filename, mode='w') as grid:
#     grid = csv.writer(grid, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#     for mean, stdev, param in zip(means, stds, params):
#       grid.writerow([mean, stdev, param])

In [ ]:
# from sklearn.externals import joblib

# # #save your model or results
# # joblib.dump(grid_result, 'gs_object_monk3.pkl')
# # #load your model for further usage
# # boh = joblib.load("gs_object_monk3.pkl")

# # joblib.dump(grid.best_estimator_, 'gs_best_estimator_monk3.pkl', compress = 1)
# # filename = 'finalized_model.sav'
# # pickle.dump(model, open(filename, 'wb'))